# Deploy Web App on Azure Container Services (AKS)


In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending an image to it and getting it scored. 

The process is split into the following steps:
- Define our resource names
- Login to Azure
- Create resource group and create AKS
- Connect to AKS
- Deploy our app
- Tear it all down

We assume that this notebook is running on Linux and Azure CLI is installed before proceeding.

## Setup

Below are the various name definitions for the resources needed to setup AKS as well as the name of the Docker image we will be using. If you wish to use the image that you previously pushed to your account, make sure you change this.

In [1]:
# Please modify the below as you see fit
resource_group = "mabouaks"
aks_name = "mabouaks"
location = "eastus"

docker_login = 'mabouatmicrosoft'  # 'fboylu/kerastf-gpu' Feel free to use this image if you want to 
image_name = docker_login + '/kerastf-gpu' # skip creating your own container

selected_subscription = "'Boston-DS-Brandon-Dev'" # If you have multiple subscriptions select 
                                                # the subscription you want to use here

## Azure account login

The command below will initiate a login to your Azure account. It will pop up with an url to go to where you will enter a one off code and log into your Azure account using your browser.

In [2]:
!az login -o table

Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"

(firefox:82222): LIBDBUSMENU-GLIB-WARNING **: Unable to get session bus: Error spawning command line 'dbus-launch --autolaunch=e05d81ec9eea4cb7abfd3fd9ec6f3655 --binary-syntax --close-stderr': Child process exited with code 1
You have logged in. Now let us find all subscriptions you have access to...
CloudName    IsDefault    Name                                State     TenantId
-----------  -----------  ----------------------------------  --------  ------------------------------------
AzureCloud   False        Boston DS Dev                       Enabled   72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   False        Solution Template Testing           Enabled   72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   False        CTS-DS                              Disabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   False        DEMO - how RepDemo are you       

In [3]:
!az account set --subscription $selected_subscription

In [4]:
!az account show

{
  "environmentName": "AzureCloud",
  "id": "e984a9db-1a27-4f54-98fc-282cf0dcda04",
  "isDefault": true,
  "name": "Boston-DS-Brandon-Dev",
  "state": "Enabled",
  "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  "user": {
    "name": "mabou@microsoft.com",
    "type": "user"
  }
}


You will also need to register the container service resources on your subscription if you haven't already done so.

In [5]:
!az provider register -n Microsoft.ContainerService

Registering is still on-going. You can monitor using 'az provider show -n Microsoft.ContainerService'


In [6]:
!az provider show -n Microsoft.ContainerService

{
  "authorization": {
    "applicationId": "7319c514-987d-4e9b-ac3d-d38c4f427f4c",
    "managedByRoleDefinitionId": "8e3af657-a8ff-443c-a75c-2fe8c4bcb635",
    "roleDefinitionId": "1b4a0c7f-2217-416f-acfa-cf73452fdc1c"
  },
  "id": "/subscriptions/e984a9db-1a27-4f54-98fc-282cf0dcda04/providers/Microsoft.ContainerService",
  "namespace": "Microsoft.ContainerService",
  "registrationState": "Registered",
  "resourceTypes": [
    {
      "aliases": null,
      "apiVersions": [
        "2017-07-01",
        "2017-01-31",
        "2016-09-30",
        "2016-03-30"
      ],
      "capabilities": "None",
      "locations": [
        "Japan East",
        "Central US",
        "East US 2",
        "Japan West",
        "East Asia",
        "South Central US",
        "Australia East",
        "Australia Southeast",
        "Brazil South",
        "Southeast Asia",
        "West US",
        "North Central US",
        "West Europe",
        "North Europe",
        "East US",
        "UK West"

## Create resources and dependencies

### Create resource group and AKS cluster

Azure encourages the use of groups to organize all the Azure components you deploy. That way it is easier to find them but also we can delete a number of resources simply by deleting the group.

In [7]:
!az group create --name $resource_group --location $location

{
  "id": "/subscriptions/e984a9db-1a27-4f54-98fc-282cf0dcda04/resourceGroups/mabouaks",
  "location": "eastus",
  "managedBy": null,
  "name": "mabouaks",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null
}


Below, we create the AKS cluster in the resource group we created earlier. This could take 5-10 minutes.

In [8]:
%%time
!az aks create --resource-group $resource_group --name $aks_name --node-count 1 --generate-ssh-keys -s Standard_NC6

{- Finished ..
  "aadProfile": null,
  "addonProfiles": null,
  "agentPoolProfiles": [
    {
      "count": 1,
      "dnsPrefix": null,
      "fqdn": null,
      "maxPods": 110,
      "name": "nodepool1",
      "osDiskSizeGb": null,
      "osType": "Linux",
      "ports": null,
      "storageProfile": "ManagedDisks",
      "vmSize": "Standard_NC6",
      "vnetSubnetId": null
    }
  ],
  "dnsPrefix": "mabouaks-mabouaks-e984a9",
  "enableRbac": true,
  "fqdn": "mabouaks-mabouaks-e984a9-91280f94.hcp.eastus.azmk8s.io",
  "id": "/subscriptions/e984a9db-1a27-4f54-98fc-282cf0dcda04/resourcegroups/mabouaks/providers/Microsoft.ContainerService/managedClusters/mabouaks",
  "kubernetesVersion": "1.9.9",
  "linuxProfile": {
    "adminUsername": "azureuser",
    "ssh": {
      "publicKeys": [
        {
          "keyData": "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDA912ocZhmAUJbpPW/mOg5anphH0ehCLNlgofEvuCCwIV/2oRBL2uC6F2fDImkOjwSLQwOdhK+IvLOBUTdBOSXwOG+Dv6kd0n+YIKrykI60UJJjx+gYw3lFqeSgkjavKQpcQO1jhne

### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [9]:
!sudo env "PATH=$PATH" az aks install-cli

Please ensure that /usr/local/bin is in your search PATH, so the `kubectl` command can be found.


## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [12]:
!az aks get-credentials --resource-group $resource_group --name $aks_name

Merged "mabouaks" as current context in /home/mabou/.kube/config


Let's verify connection by listing the nodes.

In [13]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-99663626-0   Ready     agent     9m        v1.9.9


Let's check the pods on our cluster.

In [14]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
kube-system   azureproxy-5fc9fc6976-b6kvl             1/1       Running   2          9m
kube-system   heapster-56c6f9566f-vfkqj               2/2       Running   0          9m
kube-system   kube-dns-v20-7c556f89c5-qxtxs           3/3       Running   0          9m
kube-system   kube-dns-v20-7c556f89c5-zxrq2           3/3       Running   0          9m
kube-system   kube-proxy-g2lzq                        1/1       Running   0          9m
kube-system   kube-svc-redirect-b4njg                 1/1       Running   0          9m
kube-system   kubernetes-dashboard-6c6d86c85c-sgs4w   1/1       Running   3          9m
kube-system   tunnelfront-6667d47b7f-jtsdn            1/1       Running   0          9m


## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the volume mounts to the drivers that are located on the node.

In [15]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "replicas":1,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-dl"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-dl",
                      "image": docker_login + "/kerastf-gpu",
                      "env":[
                          {
                              "name": "LD_LIBRARY_PATH",
                              "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.5/lib"
                          }
                      ],
                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "volumeMounts":[
                          {
                              "mountPath":"/usr/local/nvidia",
                              "name": "nvidia",
                          }
                      ],
                      "resources":{
                           "requests":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           },
                           "limits":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           }
                       }  
                  }
              ],
              "volumes":[
                  {
                      "name": "nvidia",
                      "hostPath":{
                          "path":"/usr/local/nvidia"
                      },
                  },
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-dl"
      }
   }
}

In [16]:
import json
def write_json_to_file(json_dict, filename, mode='w'):
    with open(filename, mode) as outfile:
        json.dump(json_dict, outfile, indent=4, sort_keys=True)
        outfile.write('\n\n')

In [17]:
write_json_to_file(app_template, 'az-dl.json')

In [18]:
write_json_to_file(service_temp, 'az-dl.json', mode='a')

Let's check the manifest created.

In [19]:
!cat az-dl.json

{
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-dl"
    },
    "spec": {
        "replicas": 1,
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-dl"
                }
            },
            "spec": {
                "containers": [
                    {
                        "env": [
                            {
                                "name": "LD_LIBRARY_PATH",
                                "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.5/lib"
                            }
                        ],
                        "image": "mabouatmicrosoft/kerastf-gpu",
                        "name": "azure-dl",
                        "ports": [
                            {
                                "containerPort": 80,
                                "name": "model"
                            }
                        ],
     

Next, we will use kubectl create command to deploy our application.

In [20]:
!kubectl create -f az-dl.json

deployment.apps/azure-dl created
service/azure-dl created


Let's check if the pod is deployed. It may take as many as 10 minutes for the container to be ready.

In [22]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-dl-57cfdc6bb4-8fpwf               1/1       Running   0          8m
kube-system   azureproxy-5fc9fc6976-b6kvl             1/1       Running   2          18m
kube-system   heapster-56c6f9566f-vfkqj               2/2       Running   0          18m
kube-system   kube-dns-v20-7c556f89c5-qxtxs           3/3       Running   0          18m
kube-system   kube-dns-v20-7c556f89c5-zxrq2           3/3       Running   0          18m
kube-system   kube-proxy-g2lzq                        1/1       Running   0          18m
kube-system   kube-svc-redirect-b4njg                 1/1       Running   0          18m
kube-system   kubernetes-dashboard-6c6d86c85c-sgs4w   1/1       Running   3          18m
kube-system   tunnelfront-6667d47b7f-jtsdn            1/1       Running   0          18m


If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [23]:
!kubectl get events

LAST SEEN   FIRST SEEN   COUNT     NAME                                         KIND         SUBOBJECT                   TYPE      REASON                    SOURCE                                 MESSAGE
20m         20m          1         aks-nodepool1-99663626-0.1540603782692417    Node                                     Normal    Starting                  kubelet, aks-nodepool1-99663626-0      Starting kubelet.
18m         20m          8         aks-nodepool1-99663626-0.15406037840ee1c8    Node                                     Normal    NodeHasSufficientDisk     kubelet, aks-nodepool1-99663626-0      Node aks-nodepool1-99663626-0 status is now: NodeHasSufficientDisk
18m         20m          8         aks-nodepool1-99663626-0.15406037840f08d8    Node                                     Normal    NodeHasSufficientMemory   kubelet, aks-nodepool1-99663626-0      Node aks-nodepool1-99663626-0 status is now: NodeHasSufficientMemory
19m         20m          7         aks-nodepool1-99663

Check the logs for the application pod.

In [24]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))

In [25]:
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

2018-07-11 17:39:43,751 CRIT Supervisor running as root (no user in config file)
2018-07-11 17:39:43,752 INFO supervisord started with pid 1
2018-07-11 17:39:44,755 INFO spawned: 'program_exit' with pid 9
2018-07-11 17:39:44,756 INFO spawned: 'nginx' with pid 10
2018-07-11 17:39:44,757 INFO spawned: 'gunicorn' with pid 11
2018-07-11 17:39:45,783 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-07-11 17:39:45.934152: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2018-07-11 17:39:46.073231: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1030] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 2e67:00:00.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-07-11 17:39:46.073275: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1120] Creating TensorFlow dev

In [26]:
!kubectl get deployment

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
azure-dl   1         1         1            1           9m


It can take a few minutes for the service to populate the EXTERNAL-IP field below. This will be the IP you use to call the service. You can also specify an IP to use, please see the AKS documentation for further details.

In [27]:
!kubectl get service azure-dl

NAME       TYPE           CLUSTER-IP     EXTERNAL-IP      PORT(S)        AGE
azure-dl   LoadBalancer   10.0.213.103   137.117.35.216   80:31203/TCP   9m


Next, we will [test our web application deployed on AKS](05_TestWebApp.ipynb). Once, we are done with all the notebooks of the tutorial, below instructions can be used to delete the cluster and free resources.

# Tear it all down

Once you are done with your cluster you can use the following two commands to destroy it all.

In [2]:
!kubectl delete -f az-dl.json

deployment.apps "azure-dl" deleted
service "azure-dl" deleted


In [3]:
!az aks delete -n $aks_name -g $resource_group -y

inished ..

In [4]:
!az group delete --name $resource_group -y

inished ..